<h1>Initial procedures and connecting to my local database</h1>

In [47]:
#1) Installed WAMP on my machine
#2) Downloaded the SQL files for the three tables (bookings, facilities and members) from country_club Database
#3) Ran the files on phpmyadmin
#4) Ran 'pip install mysql-connector-python' on anaconda's command prompt

#5) Connect to the Database

#5.1) import create_engine from sqlalchemy
#!/usr/bin/python
from sqlalchemy import create_engine
import pandas as pd

#using the mysql+mysqlconnector
engine = create_engine('mysql+mysqlconnector://root:@localhost/country_club')

#check the table_names to see if it worked
#engine.table_names()    #worked

#define a function tha connects to the engine, runs the query and returns the dataFrame
def runq(query):
    con = engine.connect()
    rs = con.execute(query)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    con.close()
    return df

#Test one query
query = 'SELECT * FROM bookings'
runq(query)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
...,...,...,...,...,...
4038,4038,8,29,2012-09-30 16:30:00,2
4039,4039,8,29,2012-09-30 18:00:00,1
4040,4040,8,21,2012-09-30 18:30:00,1
4041,4041,8,16,2012-09-30 19:00:00,1


<h2>Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.<h2/>

In [48]:
query = 'SELECT name, membercost FROM facilities WHERE membercost = 0'
runq(query)

#p.s: showing membercost column just to make sure the answer is correct

,name,membercost
0,Badminton Court,0.0
1,Table Tennis,0.0
2,Snooker Table,0.0
3,Pool Table,0.0


<h2>Q2: How many facilities do not charge a fee to members?</h2>

In [49]:
query = 'SELECT count(name) FROM facilities WHERE membercost = 0'
runq(query)

,count(name)
0,4


<h2>Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.</h2>

In [50]:
query = 'SELECT facid, name, membercost, monthlymaintenance FROM facilities WHERE membercost <= 0.2*monthlymaintenance' 
query += ' AND membercost > 0' 
runq(query)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


<h2>Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator.</h2>

In [51]:
query = 'SELECT * FROM facilities WHERE facid in (1,5)'
runq(query)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


<h2>Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.</h2>

In [52]:
#Can't use column alias on WHERE clause, so I made one subquery to create the price_category, then one outer 
#query to filter the results I wanted (just the expensive labels)

query = 'SELECT * FROM '
query += '(SELECT name, monthlymaintenance, ' 
query += '(CASE WHEN monthlymaintenance > 100 THEN "expensive" '
query +=    'ELSE "cheap" END) as price_category '
query += 'FROM facilities) AS aux ' 
query += 'WHERE aux.price_category = "expensive"'
runq(query)

,name,monthlymaintenance,price_category
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Massage Room 1,3000,expensive
3,Massage Room 2,3000,expensive


<h2>Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.</h2>

In [53]:
#on Table "members", check the most recent joindate

query = 'SELECT firstname, surname FROM members WHERE joindate = (SELECT max(joindate) FROM members)'
runq(query)

#the alternative would be to list DESC by joinDate and limit the result to 1, but LIMIT is not allowed

,firstname,surname
0,Darren,Smith


<h2>Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.</h2>

In [57]:
query = 'SELECT DISTINCT f.name as Court_name, CONCAT(m.firstname," ", m.surname) as Member_name '
query += 'FROM bookings AS b '
query += 'JOIN members AS m '
query += 'ON b.memid = m.memid '
query += 'JOIN facilities as f '
query += 'ON b.facid = f.facid '
query += 'AND f.name LIKE "Tennis Court%" '
query += 'ORDER BY Member_name'

runq(query)

,Court_name,Member_name
0,Tennis Court 2,Anne Baker
1,Tennis Court 1,Anne Baker
2,Tennis Court 1,Burton Tracy
3,Tennis Court 2,Burton Tracy
4,Tennis Court 2,Charles Owen
5,Tennis Court 1,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 2,David Farrell
8,Tennis Court 1,David Farrell
9,Tennis Court 1,David Jones


<h2>Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.</h2>

In [58]:
query =  'SELECT f.name as Facility_name, CONCAT(m.firstname," ", m.surname) as Member_name, '
query +=  '(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost '
query +=  'ELSE b.slots * f.membercost END) as Cost '
query +=  'FROM `bookings` as b '
query +=  'JOIN facilities as f '
query +=  'ON b.facid = f.facid '
query +=  'JOIN members as m '
query +=  'ON b.memid = m.memid '
query +=  'WHERE starttime LIKE "2012-09-14%" '
query +=  'AND (CASE WHEN b.memid = 0 THEN b.slots * f.guestcost '
query +=  'ELSE b.slots * f.membercost END) > 30 '
query +=  'ORDER BY Cost DESC '

runq(query)

,Facility_name,Member_name,Cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 2,GUEST GUEST,75.0
7,Tennis Court 1,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


<h2>Q9: This time, produce the same result as in Q8, but using a subquery.</h2>

In [63]:
query =   'SELECT * FROM '
query +=  '(SELECT f.name as Facility_name, CONCAT(m.firstname," ", m.surname) as Member_name, '
query +=  '(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost '
query +=  'ELSE b.slots * f.membercost END) as Cost '
query +=  'FROM `bookings` as b '
query +=  'JOIN facilities as f '
query +=  'ON b.facid = f.facid '
query +=  'JOIN members as m '
query +=  'ON b.memid = m.memid '
query +=  'WHERE starttime LIKE "2012-09-14%") as Aux '
query +=  'WHERE Aux.Cost > 30 '
query +=  'ORDER BY Aux.Cost DESC'

runq(query)

,Facility_name,Member_name,Cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 2,GUEST GUEST,75.0
7,Tennis Court 1,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


<h2>Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!</h2>

In [65]:
query =   'SELECT Aux.name as Facility_name, SUM(Aux.rev) as Total_Revenue FROM '
query +=  '(SELECT f.name, '
query +=  ' (CASE WHEN b.memid = 0 THEN b.slots * f.guestcost '
query +=  ' ELSE b.slots * f.membercost END) as rev '
query +=  'FROM `bookings` as b '
query +=  'JOIN facilities as f '
query +=  'ON b.facid = f.facid) AS Aux '
query +=  'GROUP BY Aux.name '
query +=  'HAVING Total_Revenue <1000 '
query +=  'ORDER BY Total_Revenue DESC '

runq(query)

,Facility_name,Total_Revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0
